---
# Projectr : 하루시작 지하철 혼잡도 분석
### Description : 
- <span style = "color : #FFBE98">서울교통공사 역사별 위・경도 변환하기 </span>
    
### Author : Zenzen
### Date : 2024. 06. 03. (Mon)
### Detail : 
### Update: 
    * 2024.06.03 zenzen : Kakao Open API(REST API)를 통한 위・경도 변환하기.
    * 2024.06.09 by pdg :  station info  수정한 버전으로 다시 위도경도 변환 
        - 칼럼수가 바뀌어서 저장 잘못됨 -> 수정 


---

In [1]:
import warnings

warnings.filterwarnings('ignore')

# Data 정제

## Kakao Open API 이용하여 위・경도 반환하기

In [2]:
import pandas as pd

seoul_subway = pd.read_csv("../Data/StationInfo.csv")

seoul_subway.head()

,역사코드,역명,호선,환승역수
0,150,서울역,1,2
1,426,서울역,4,2
2,151,시청,1,2
3,201,시청,2,2
4,152,종각,1,1


### '역명' Column Data 중에서 끝자리가 '역' 단어로 끝나는 역명 검색하기.

In [3]:
# import re

# pattern = r'\w*역$'  # 임의의 문자(단어 문자)가 0번 이상 반복되고 '역'으로 끝나는 패턴

# matches = seoul_subway['역명'].str.findall(pattern)

# seoul_subway[seoul_subway['역명'].str.contains(pattern)]

In [8]:
import requests
import re

# 괄호 안 내용 제거
seoul_subway['역명'] = seoul_subway['역명'].apply(lambda x: re.sub(r'\([^)]*\)', '', x) if '(' in x else x)

# '역명' Column Data 중에서 끝자리에 '역' 단어가 없는 경우에만 '역'을 추가
seoul_subway['역명'] = seoul_subway['역명'].apply(lambda x: x + '역' if not x.endswith('역') else x)

x = []  # List of latitude
y = []  # List of longitude

for z in seoul_subway['역명']:
    url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(z)
    headers = {
        "Authorization": "KakaoAK bac1c627e6f438d7e8f566dcca6d8764"
    }

    # JSON 형태로 데이터가 주어진다
    places = requests.get(url, headers=headers).json()['documents'][0]
    # print(places)
    x.append(places['y'])
    y.append(places['x'])

seoul_subway['latitude'] = x
seoul_subway['longitude'] = y
seoul_subway_latlon = seoul_subway.reset_index()

seoul_subway_latlon.head()

{'address_name': '서울 용산구 동자동 43-205', 'category_group_code': '', 'category_group_name': '', 'category_name': '교통,수송 > 기차,철도 > 기차역 > KTX정차역', 'distance': '', 'id': '9113903', 'phone': '1544-7788', 'place_name': '서울역', 'place_url': 'http://place.map.kakao.com/9113903', 'road_address_name': '서울 용산구 한강대로 405', 'x': '126.970606917394', 'y': '37.5546788388674'}
{'address_name': '서울 용산구 동자동 43-205', 'category_group_code': '', 'category_group_name': '', 'category_name': '교통,수송 > 기차,철도 > 기차역 > KTX정차역', 'distance': '', 'id': '9113903', 'phone': '1544-7788', 'place_name': '서울역', 'place_url': 'http://place.map.kakao.com/9113903', 'road_address_name': '서울 용산구 한강대로 405', 'x': '126.970606917394', 'y': '37.5546788388674'}
{'address_name': '서울 중구 정동 5-5', 'category_group_code': 'SW8', 'category_group_name': '지하철역', 'category_name': '교통,수송 > 지하철,전철 > 수도권1호선', 'distance': '', 'id': '21160570', 'phone': '02-6110-1321', 'place_name': '시청역 1호선', 'place_url': 'http://place.map.kakao.com/21160570', 'road_addr

,index,역사코드,역명,호선,환승역수,latitude,longitude
0,0,150,서울역,1,2,37.5546788388674,126.970606917394
1,1,426,서울역,4,2,37.5546788388674,126.970606917394
2,2,151,시청역,1,2,37.56534539636417,126.97719821079865
3,3,201,시청역,2,2,37.56534539636417,126.97719821079865
4,4,152,종각역,1,1,37.570227990912244,126.98315081716676


In [17]:
## 칼럼수 변경됨 
seoul_subway_latlon_2 = seoul_subway_latlon.iloc[:, 1:]

In [18]:
seoul_subway_latlon_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 279 entries, 0 to 278
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   역사코드       279 non-null    int64 
 1   역명         279 non-null    object
 2   호선         279 non-null    int64 
 3   환승역수       279 non-null    int64 
 4   latitude   279 non-null    object
 5   longitude  279 non-null    object
dtypes: int64(3), object(3)
memory usage: 13.2+ KB


In [19]:
seoul_subway_latlon_2['latitude'] = seoul_subway_latlon_2['latitude'].astype(float)
seoul_subway_latlon_2['longitude'] = seoul_subway_latlon_2['longitude'].astype(float)

In [20]:
seoul_subway_latlon_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 279 entries, 0 to 278
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   역사코드       279 non-null    int64  
 1   역명         279 non-null    object 
 2   호선         279 non-null    int64  
 3   환승역수       279 non-null    int64  
 4   latitude   279 non-null    float64
 5   longitude  279 non-null    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 13.2+ KB


In [21]:
seoul_subway_latlon_2.to_csv("../Data/seoul_subway_latlon_zenzen.csv", index = False, encoding = 'utf-8')

In [22]:
pd.read_csv("../Data/seoul_subway_latlon_zenzen.csv")

,역사코드,역명,호선,환승역수,latitude,longitude
0,150,서울역,1,2,37.554679,126.970607
1,426,서울역,4,2,37.554679,126.970607
2,151,시청역,1,2,37.565345,126.977198
3,201,시청역,2,2,37.565345,126.977198
4,152,종각역,1,1,37.570228,126.983151
...,...,...,...,...,...,...
274,2649,신내역,6,1,37.612491,127.104271
275,2758,상동역,7,1,37.505789,126.753192
276,2761,부평구청역,7,1,37.507434,126.721440
277,2753,까치울역,7,1,37.506227,126.811340
